In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json

In [2]:
f = open("./uber-eats-popular-categories.txt")
lines = f.read().split('\n')
categories = set(lines)
if '' in categories:
    categories.remove('')
len(categories)
categories

51

{'Allergy Friendly',
 'American',
 'Asian',
 'Asian Fusion',
 'BBQ',
 'Bakery',
 'Breakfast and Brunch',
 'Bubble Tea',
 'Burgers',
 'Burritos',
 'Cafe',
 'Chicken',
 'Chinese',
 'Coffee and Tea',
 'Comfort Food',
 'Cupcakes',
 'Curry',
 'Deli',
 'Desserts',
 'Donuts',
 'Fast Food',
 'Greek',
 'Healthy',
 'Ice Cream',
 'Indian',
 'Italian',
 'Japanese',
 'Juice and Smoothies',
 'Korean',
 'Latin American',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Noodles',
 'Pasta',
 'Pastry',
 'Pizza',
 'Ramen',
 'Rice',
 'Salads',
 'Sandwich',
 'Seafood',
 'Steak',
 'Sushi',
 'Tacos',
 'Thai',
 'Vegan',
 'Vegetarian',
 'Vietnamese',
 'Wings',
 'Wraps'}

In [3]:
from urllib.error import URLError
import time

def get_city_rests(city_url, categories, debug=True):
    start_time = time.time()
    rests = set()
    categ_stats = {}
    
    for category in categories:
        category = category.lower().replace(" ", "-")
        category_city_url = "https://www.ubereats.com/category/" + city_url.split('/')[-1] + "/" + category
        try:
            req = Request(category_city_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})

            webpage = urlopen(req).read()
            soup = BeautifulSoup(webpage, 'html.parser')
            print(soup.title)
            rest_links = soup.find_all("h3", class_="dr bn bo at ep al aj eq dp")
            categ_stats[category] = len(rest_links)
            old_rests_size = len(rests)

            if debug == True:
                print(f"found: {len(rest_links)}, old size: {old_rests_size}")
            for link in rest_links:
                rests.add(link.a['href'])
            if debug == True:
                print(f"added: {len(rests)-old_rests_size}, new size: {len(rests)}")
        except Exception as e:
            print(f"\nerror: {e}\n url: {category_city_url}")
            categ_stats[category] = 0
    elapsed_time = time.time()-start_time
    print(f"time elapsed: {elapsed_time}")
    return rests, categ_stats

In [4]:
city_url = "https://www.ubereats.com/location/new-york"

In [5]:
city_rests, categ_stats = get_city_rests(city_url, categories, debug=True)

<title>Asian Delivery | New York City | Uber Eats</title>
found: 419, old size: 0
added: 419, new size: 419
<title>Italian Delivery | New York City | Uber Eats</title>
found: 0, old size: 419
added: 0, new size: 419
<title>Korean Delivery | New York City | Uber Eats</title>
found: 55, old size: 419
added: 9, new size: 428
<title>Pastry Delivery | New York City | Uber Eats</title>
found: 28, old size: 428
added: 28, new size: 456
<title>Asian Fusion Delivery | New York City | Uber Eats</title>
found: 142, old size: 456
added: 0, new size: 456
<title>Middle Eastern Delivery | New York City | Uber Eats</title>
found: 37, old size: 456
added: 37, new size: 493
<title>Latin American Delivery | New York City | Uber Eats</title>
found: 74, old size: 493
added: 74, new size: 567
<title>Pasta Delivery | New York City | Uber Eats</title>
found: 171, old size: 567
added: 166, new size: 733
<title>Vietnamese Delivery | New York City | Uber Eats</title>
found: 34, old size: 733
added: 20, new size:

In [6]:
sorted(categ_stats.items(), key = lambda x : x[1], reverse=True)

[('salads', 852),
 ('american', 689),
 ('desserts', 609),
 ('burgers', 525),
 ('sandwich', 476),
 ('asian', 419),
 ('vegetarian', 322),
 ('comfort-food', 288),
 ('chicken', 287),
 ('breakfast-and-brunch', 267),
 ('seafood', 205),
 ('japanese', 185),
 ('pasta', 171),
 ('coffee-and-tea', 171),
 ('chinese', 162),
 ('pizza', 156),
 ('asian-fusion', 142),
 ('tacos', 134),
 ('sushi', 130),
 ('mexican', 127),
 ('juice-and-smoothies', 121),
 ('vegan', 120),
 ('burritos', 108),
 ('donuts', 105),
 ('wraps', 98),
 ('fast-food', 90),
 ('wings', 87),
 ('steak', 84),
 ('rice', 83),
 ('bakery', 78),
 ('latin-american', 74),
 ('indian', 70),
 ('thai', 64),
 ('cafe', 58),
 ('korean', 55),
 ('cupcakes', 50),
 ('bubble-tea', 48),
 ('deli', 46),
 ('bbq', 44),
 ('ramen', 40),
 ('middle-eastern', 37),
 ('curry', 37),
 ('greek', 36),
 ('vietnamese', 34),
 ('ice-cream', 32),
 ('pastry', 28),
 ('italian', 0),
 ('mediterranean', 0),
 ('healthy', 0),
 ('allergy-friendly', 0),
 ('noodles', 0)]

In [19]:
import pandas as pd
df = pd.DataFrame(columns=['url', 'city'])
i=0
city_name = city_url.split('/')[-1]
for rest in city_rests:
    df.loc[i] = [
        'https://www.ubereats.com' + rest,
        city_name
    ]
    i = i+1

In [23]:
df.head()

,url,city
0,https://www.ubereats.com/new-york/food-deliver...,new-york
1,https://www.ubereats.com/new-york/food-deliver...,new-york
2,https://www.ubereats.com/new-york/food-deliver...,new-york
3,https://www.ubereats.com/new-york/food-deliver...,new-york
4,https://www.ubereats.com/new-york/food-deliver...,new-york


In [24]:
df.to_csv('./UE-NY-rests-list.csv', index=False)

In [ ]:
def get_rest_details(rest_url):
    start_time = time.time()
    rest_obj = {}
    try:
        req = Request(rest_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        rest_obj = json.loads(soup.find("script", type="application/ld+json").text)
        print(rest_obj['name'])
    except Exception as e:
        print(f"\nerror: {e}\n url: {rest_url}")
    
    elapsed_time = time.time()-start_time
    print(f"elapsed time: {elapsed_time}")
    return rest_obj

In [ ]:
rest_url = "https://www.ubereats.com/new-york/food-delivery/cha-chan-tang/CKWqFu2RSeK7rxMdN75tpA"
rest_obj = get_rest_details(rest_url)
rest_obj